In [ ]:
# pip install langchain langchain-google-genai langchain-neo4j
# pip install openai

In [1]:
import os
from uuid import uuid4

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_neo4j import Neo4jChatMessageHistory, Neo4jGraph


In [2]:
NEO4J_URI = "<NEO4j_URI>"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "<API KEY>"

In [55]:

# Initialize session
session_id = str(uuid4())
print(f"Session ID: {session_id}")

# Load environment variables once
google_api_key = "<API KEY>" #os.environ.get("GOOGLE_API_KEY")
neo4j_uri = NEO4J_URI #os.environ.get("NEO4J_URI")
neo4j_user = NEO4J_USER #os.environ.get("NEO4J_USERNAME")
neo4j_pass = NEO4J_PASSWORD #os.environ.get("NEO4J_PASSWORD")

# Initialize Google Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=google_api_key,
    temperature=0.2
)

# Initialize Neo4j
graph = Neo4jGraph(url=neo4j_uri, username=neo4j_user, password=neo4j_pass)

# Define prompt
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "Your job is to convert user input into a cypher query.Strictly respond only with a cypher query."),
#     ("human", "Create cypher query using these contents: {input}")
# ])
graph_schema_system_prompt = """In order for you to write cypher queries the following is the graph schema you need. Strictly stick to naming conventions and ontologies in the schema""" + str(graph.schema.replace('{','{{').replace('}','}}'))
prompt = ChatPromptTemplate.from_messages([    
    ("system", graph_schema_system_prompt),
    ("system", "Your job is to convert user input into a cypher query. Strictly respond only with a cypher query. Do not include any text except the generated Cypher statement."),
    ("human", "Create cypher query using these contents: {input}"),
])

# Chain LLM with prompt
cypher_chat = prompt | llm | StrOutputParser()

# # Define memory
def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

# Tool definition
tools = [
    Tool.from_function(
        name="Cypher Support",
        description="All things related to cypher queries Cypher queries or create cypher queries.",
        func=lambda x: cypher_chat.invoke({"input": x}),
    )
]

# Agent setup
agent_prompt = hub.pull("hwchase17/react-chat")
# agent_prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

cypher_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)
'''
We are not leveraging the neo4j for conversation history preservation here. 
The code is hear as placeholders I can have only one free neo4j cloud instance active at a time.
'''
# Main interaction loop
while True:
    try:
        user_input = input("> ").strip()
        if user_input.lower() in ("exit", "quit"):
            print("Exiting...")
            break
        more_context_input = "write and find the cypher query equivalent of this : "+ user_input
        response = cypher_agent.invoke({"input": str(more_context_input)},
                                       {"configurable": {"session_id": session_id}})
        print(response.get("output", "[No output]"))

    except KeyboardInterrupt:
        print("\nSession ended by user.")
        break
    except Exception as e:
        print(f"Error: {e}")
        break

Session ID: e02c7de2-ef1e-4c5c-9a7a-dc573cba6acd


C:\Users\krish\anaconda3\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


>  plans pricing and all kinds of fees


Here is the Cypher query that retrieves plans, their pricing, and all kinds of fees:

```cypher
MATCH (p:Plan)
OPTIONAL MATCH (p)-[:HAS_PRICING]->(pricing:Pricing)
OPTIONAL MATCH (p)-[:HAS_FEE]->(fee:Fee)
OPTIONAL MATCH (p)-[:HAS_ONE_TIME_FEE]->(oneTimeFee:Fee)
OPTIONAL MATCH (p)-[:HAS_EARLY_TERMINATION_FEE]->(earlyTerminationFee:Fee)
OPTIONAL MATCH (p)-[:HAS_FEDERAL_FEE]->(federalFee:Fee)
OPTIONAL MATCH (p)-[:HAS_REGULATORY_FEE]->(regulatoryFee:Fee)
OPTIONAL MATCH (p)-[:HAS_TAX]->(tax:Fee)
RETURN p, pricing, fee, oneTimeFee, earlyTerminationFee, federalFee, regulatoryFee, tax
```


>  quit


Exiting...


In [ ]:
### Pipeline is fine - need to use seperate graph for conversation history and seperate for plans and domain details etc

In [ ]:
parsed_cq = response["output"].split("cypher\n")[1].split("\n")[0] # Parsing needs t be optimised for a output structure

In [57]:
parsed_cq = '''MATCH (p:Plan)
OPTIONAL MATCH (p)-[:HAS_PRICING]->(pricing:Pricing)
OPTIONAL MATCH (p)-[:HAS_FEE]->(fee:Fee)
OPTIONAL MATCH (p)-[:HAS_ONE_TIME_FEE]->(oneTimeFee:Fee)
OPTIONAL MATCH (p)-[:HAS_EARLY_TERMINATION_FEE]->(earlyTerminationFee:Fee)
OPTIONAL MATCH (p)-[:HAS_FEDERAL_FEE]->(federalFee:Fee)
OPTIONAL MATCH (p)-[:HAS_REGULATORY_FEE]->(regulatoryFee:Fee)
OPTIONAL MATCH (p)-[:HAS_TAX]->(tax:Fee)
RETURN p, pricing, fee, oneTimeFee, earlyTerminationFee, federalFee, regulatoryFee, tax'''

In [59]:
def execute_cypher_query(graph: Neo4jGraph, cypher_query: str):
    try:
        result = graph.query(cypher_query)
        return result
    except Exception as e:
        return f"Error executing Cypher query: {e}"

In [61]:
query_result = execute_cypher_query(graph, parsed_cq)

In [62]:
query_result

[{'p': {'monthly_price': '$40.00',
   'disclosure_title': 'Broadband Consumer Label',
   'name': 'Broadband Unlimited Talk and Text with 20GB of Data',
   'data_included': '20 GB',
   'additional_data_charges': 'Check terms and conditions'},
  'pricing': None,
  'fee': None,
  'oneTimeFee': {'amount': 'Varies', 'name': 'One-Time Fees at Purchase'},
  'earlyTerminationFee': {'amount': '$0.00', 'name': 'Early Termination Fee'},
  'federalFee': {'amount': '$0.23',
   'name': 'Federal Universal Service Fund (FUSF)'},
  'regulatoryFee': {'amount': '$0.08', 'name': 'Regulatory Cost Recovery'},
  'tax': {'amount': 'Varies by location', 'name': 'Government Taxes'}},
 {'p': {'monthly_price': '$40.00',
   'disclosure_title': 'Broadband Consumer Label',
   'name': 'Broadband Unlimited Talk and Text with 20GB of Data',
   'data_included': '20 GB',
   'additional_data_charges': 'Check terms and conditions'},
  'pricing': None,
  'fee': None,
  'oneTimeFee': {'amount': '$0.00', 'name': 'Activation F